# Plant Monitoring System  
---

## Aim

* This application notebook uses Grove temperature and Grove water sensors to create a temperature and moisture monitor for plants. The conditions are monitored on the Grove led stick and Grove buzzer. An alert is generated if the water sensor does not detect wetness and if the temperature sensor capture temperature outside set limits. User will be able to change the temperature threshold limits and modify display/signal attributes like led colors and buzzer notes. The notebook can be extended to add a python based emailing system when conditions not conducive for plants are encountered.

## References
* [Jupyterplot](https://lvwerra.github.io/jupyterplot/)
* [Grove water_sensor](https://www.seeedstudio.com/Grove-Water-Sensor.html) 
* [Grove temperature](https://www.seeedstudio.com/Grove-Temperature-Sensor.html) 
* [Grove buzzer](https://www.seeedstudio.com/Grove-buzzer.html)   
* [Grove Base Shield V2.0](https://www.seeedstudio.com/Base-Shield-V2.html)   

## Last revised
* 02 April 2021
    + Initial version
---

## Load _base_ Overlay

In [ ]:
from time import sleep

from IPython.display import display, HTML
from pynq.overlays.base import BaseOverlay
from pynq_peripherals import ArduinoSEEEDGroveAdapter

base = BaseOverlay('base.bit')

<div class="alert alert-box alert-info">
   <h4 class="alert-heading">Library Dependencies </h4>
Before proceeding the following libraries must be installed
    <ul>
        <li> Jupyterplot </li>
    </ul>
</div>

In [ ]:
try:
    from jupyterplot import ProgressPlot
except ImportError:
    print("This script requires the jupyterplot module \nInstall with: sudo "
         "pip3 install jupyterplot")

## Constructing application with Grove Base Shield V2.0 (Arduino)

<div class="alert alert-box alert-danger">
    <h4 class="alert-heading">Stop before proceeding further! </h4>
     Please make sure you had connected all the sensors before switching the board ON. If the sensors are connected while the board is ON it can cause the system to shutdown. For safe use, you can note down the connections shown below, switch the board OFF make the physical connections and the switch the board back ON.
    
   <h4 class="alert-heading">Make Physical Connections </h4>
    <ul>
        <li>Insert the Grove Base Shield into the Arduino connector on the board.</li>
        <li>Connect the grove_temperature module to A1 connector of the Grove Base Shield.</li>
        <li>Connect the grove_water_sensor module to D5 connector of the Grove Base Shield.</li>
        <li>Connect the grove_led_stick module to D7 connector of the Grove Base Shield.</li>
        <li> Connect the grove_buzzer module to D6 connector of the Grove Base Shield.</li></ul>
</div>

![](images/dry_arduino.png)

### Adapter configuration

In [ ]:
adapter = ArduinoSEEEDGroveAdapter(base.ARDUINO, D7='grove_led_stick', D5='grove_water_sensor', A1='grove_temperature', D6='grove_buzzer' )

### Define device objects

In [ ]:
led_stick = adapter.D7
water_sensor = adapter.D5
temp_sensor = adapter.A1
buzzer = adapter.D6

### Construct the Temperature and Moisture Monitor

In [ ]:
led_colors = [0x0000FF, 0x00E4FF, 0x00FF5C, 0x00FF10, 0x3EFF00, 0xFFFA00,
              0xFFC800, 0xFF4600, 0xFF2800, 0xFF0000]  # Blue --> Green --> Red


def led_meter(level):
    for i in range(level):
        led_stick.set_pixel(i, led_colors[i])
    for j in range(9, level - 1, -1):
        led_stick.set_pixel(j, 0)
    led_stick.show()


def led_flash(color):
    for i in range(10):
        led_stick.set_pixel(i, color)
    led_stick.show()


def enable_buzzer():
    if base.switches[0].read():
        buzzer.play_note('g', 500)
    else:
        pass


def alert(temp_level, dry):
    if not dry:
        led_meter(temp_level)
    else:
        led_flash(0xFF0000)
        enable_buzzer()


def html_display(temp_value, dry_value):
    if dry_value:
        color = "#DC143C"
        status = " Dry !!"
    else:
        color = "#3CB371"
        status = " Wet !!"
    return f"""
            <h4>Temperature: <b style="color:#00008B"> {temp_value}°C</b></h4>
            <h4>Moisture status: <b style="color:{color}">{status}</b></h4>
            <br>
           """

<div class="alert alert-box alert-info">
   <h4 class="alert-heading">Notes before running the next cell </h4>
    <ul>
        <li> Toggle Switch SW0 on the PYNQ-Z2 board is used to enable or disable the alert buzzer.</li>
        <li> Since the Water sensor is checking for soil moisture, if the sensor is dry the buzzer will be ON as soon as you run the cell.</li>
        <li> You can use a damp tissue and place it on the water sensor with good contact to simulate the soil moisture.</li>
        <li> The Temerature sensor is sensitive enough to measure change when a finger is placed on it.</li>
    </ul>
</div>

In [ ]:
# Construct a list of 10 values ranging from room+5°C --> room --> room-5°C
limit = 5  # Set limit to +/- 5°C
rt = round(temp_sensor.get_temperature(), 2)  # Room temperature
high_points = [round((i * (limit / 6) + rt), 2) for i in range(1, 7)]
low_points = [round((rt - i * (limit / 6)), 2) for i in range(4, 0, -1)]
points = low_points + [rt] + high_points

led_meter(4)  # Set Led Stick to room temperature

hdisplay = display(HTML(''), display_id=True)  # Initialize display variable

pp = ProgressPlot(line_names=["Temperature", "High Limit", "Low Limit"],
    x_lim=[0, 200], y_lim=[rt - 2*limit, rt + 2*limit])

for i in range(200):
    temperature = round(temp_sensor.get_temperature(), 2)
    moisture = water_sensor.is_dry()
    hdisplay.update(HTML(html_display(temperature, moisture)))

    if rt + limit >= temperature >= rt - limit:
        for k, j in enumerate(points):
            if (j - (limit / 6)) < temperature < (j + (limit / 6)):
                alert(k, moisture)
    elif temperature > rt + limit:
        enable_buzzer()
        alert(10, moisture)
    else:
        enable_buzzer()
        alert(1, moisture)

    pp.update([[temperature, rt + limit, rt - limit]])
    sleep(0.5)

pp.finalize()
led_stick.clear()

Copyright (C) 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----